In [14]:
!pip install transformers
!pip install torch
!pip install -U datasets

In [1]:
import pandas as pd
import numpy as np
import torch
from tqdm.notebook import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

Convert the dataset text to a csv:

In [3]:
text_filepath = "../home/schen9/dataset.txt"
csv_filepath = "dataset.csv"

In [36]:
import csv

with open(text_filepath, "r", encoding="utf-8") as text_file:
    rows = text_file.read().strip().split("\n\n")

with open(csv_filepath, "w", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Text"])
    for row in tqdm(rows):
        writer.writerow([row.strip()])

print(f"Finished writing to {csv_filepath}") 

  0%|          | 0/449919 [00:00<?, ?it/s]

Finished writing to dataset.csv


In [37]:
from datasets import Dataset, load_dataset

dataset = load_dataset("csv",data_files=csv_filepath) 
dataset

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Text'],
        num_rows: 449919
    })
})

In [11]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

transformer_name = "FacebookAI/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(transformer_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [39]:
def tokenize(data):
    return tokenizer(data['Text'], padding="max_length", truncation=True)

batch_size = 20
tokens = dataset.map(tokenize, batched=True, batch_size=batch_size)
tokens

Map:   0%|          | 0/449919 [00:00<?, ? examples/s]

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [6]:
# Read the giant textfile in
filepath = "../home/schen9/dataset.txt"

def read_file(filepath,num_lines=850_000):
    # text_lines = []
    text = ""
    with open(filepath) as fh:
        for i, line in enumerate(fh):
            if i >= num_lines:
                break
            # text_lines.append(line)
            text += line
    # return text_lines
    return text

text = read_file(filepath)

In [ ]:
tokens = tokenizer(text, truncation=True, return_tensors='pt')

In [6]:
tokens

{'input_ids': tensor([[    0,   133,   735, 34546,    16,    10, 36248,   470,  8454,  4149,
           822,     6,  3660,    30,  4720, 11998,     8,  8996,  4810,  1587,
          1127,   338,     6,  1813, 11549,     6,     8,  4858,  8804,     4,
         50118,   243,    21,   703,    30,  1234,   496, 12290,    15,   502,
           262,     6, 36248,     4, 50118, 48812, 50118,  1620,  1602,    11,
            10,   822,  4320,  1551,     6, 12602,   710,     6,  1354,     9,
           407, 12336, 18095,   219,   859,     6,  4401,  4458,   988,  4436,
             6,  1060,   275,  1441,  3884,   506, 10505, 15071,     6,  2730,
             6, 25760,     6,     8, 33489,     6,    16,    67,    11,   657,
            19,    69,     4, 50118,   894,  2215,    14, 12602,   710,  2997,
           988,   396,   143,   657,    13,   123,     4, 50118,  1779, 12602,
           710, 16766,     7,  1157,   123,     7,   989,   928,     6,    37,
         11703,    14,    37,    34,  

In [8]:
model(**tokens)

MaskedLMOutput(loss=None, logits=tensor([[[37.5077, -3.9347, 15.3525,  ...,  3.8584,  5.4767, 12.8930],
         [ 7.3782, -2.7824, 13.6218,  ...,  4.8712,  5.0904,  8.2832],
         [ 1.1484, -3.6915,  7.2563,  ..., -0.1078, -1.0249,  3.4536],
         ...,
         [ 0.2669, -4.2053,  7.0668,  ..., -5.6054, -5.8515,  2.2009],
         [-1.2186, -3.4854, 15.0709,  ...,  2.3310, -3.1703,  4.0132],
         [ 8.2976, -3.9727, 28.2076,  ...,  1.5808, -2.9764,  8.9007]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)